# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [96]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD




In [97]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [98]:
data = pd.read_csv('/content/gdrive/MyDrive/creditcard.csv')

In [99]:
data.shape

(284807, 31)

In [100]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [101]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Data Shuffling

In [102]:
data = data.sample(frac=1).reset_index(drop=True)

In [103]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,78983.0,0.260973,-2.313863,-0.539756,-0.228495,-1.058129,0.374417,0.162362,0.036478,-1.013756,0.481190,1.290213,0.448564,-0.636163,0.543023,-0.088493,-2.087063,0.685033,0.384421,-1.060248,0.466992,-0.032221,-0.681389,-0.410288,-0.221058,0.037816,1.079924,-0.143544,0.070854,531.68,0
1,149521.0,1.971029,0.101474,-1.803507,0.535114,0.240016,-1.407008,0.385872,-0.439940,0.488870,-0.515656,-0.360613,0.391114,0.734392,-0.866036,0.837714,0.219148,0.304282,0.471777,-0.397766,-0.054257,0.224440,0.745347,-0.091432,-0.152259,0.278290,-0.102030,-0.007457,-0.022344,53.75,0
2,45375.0,1.216888,0.574737,-0.487968,1.170599,0.190012,-0.761536,0.116609,-0.005745,-0.137257,-0.475207,1.314610,-0.205720,-1.347351,-0.908439,0.551263,0.947604,0.596404,1.349628,-0.203743,-0.157075,-0.052103,-0.173230,-0.161466,-0.164674,0.662165,-0.320274,0.022758,0.041287,1.00,0
3,164256.0,2.027015,-0.600922,-1.052655,0.737600,-0.569147,-0.785292,-0.248199,-0.275615,-0.481998,1.051463,-1.256481,-0.165462,-0.023617,0.179691,-0.071862,-1.779771,-0.117805,1.315920,-1.359493,-0.616621,-0.017349,0.555889,-0.020392,-0.030081,0.258455,-0.313771,0.034163,-0.040582,50.00,0
4,163185.0,-1.660659,1.898361,-0.736685,-0.878018,-0.439088,-0.993323,1.151126,0.014037,1.167352,1.395377,-0.983832,-0.108543,-0.526268,-0.110363,-0.221549,-0.076261,-0.409069,-0.747064,-0.227263,0.485431,-0.530648,-0.731751,0.175891,-0.059957,-0.175114,0.154409,0.678029,0.161412,89.99,0


### Splitting Training and Labels

In [104]:
# Separating Input and Output Colmumns 
x = data.iloc[:, :-1].values  # Input Columns
y = data.iloc[:, -1:].values # Output Columns 


In [105]:
x.shape

(284807, 30)

In [106]:
y.shape

(284807, 1)

In [107]:
# Installed imbalanced-learn in order to use SMOTE  
# pip install -U imbalanced-learn

## As the data is very imbalanced, I have used SMOTE (Synthetic Minority Oversampling Technique) for oversampling

In [108]:
np.ravel(y)
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 42)

x, y = sm.fit_resample(x, y)


print (x.shape, y.shape)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(568630, 30) (568630,)


In [109]:
x = pd.DataFrame(x)
y = pd.DataFrame(y)
x.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
dtype: int64

### Splitting Train, Test, Validation Data

In [110]:
def split_train_val_test(x, y, train, val, test):
    x_rows, x_col = x.shape
    y_rows = y.shape # Removed column temporarily 
    
    train_start = 0
    train_stop = int(x_rows*train)
    
    val_start = train_stop
    val_stop = int(x_rows*train)+int(x_rows*val)
    
    test_start = val_stop
    test_stop = None

    x_train = x.iloc[train_start:train_stop,] # Extracting X_Training Data 
    x_val = x.iloc[val_start:val_stop,]
    x_test = x.iloc[test_start:test_stop]
    
    y_train = y.iloc[train_start:train_stop,] # Extracting y_Training Data 
    y_val = y.iloc[val_start:val_stop,]
    y_test = y.iloc[test_start:test_stop]
    
    #Splitting Desicription 
    desc = f'''Splitting Discription:
    Training Data {int(train*100)}%
    Validation Data {int(val*100)}%
    Testing Data {int(val*100)}%
    
    Shape of Training Data {x.shape}
    Shape of Target Data {y.shape}
    
    Shape of x_train {x_train.shape}
    Shape of x_val {x_val.shape}
    Shape of x_test {x_test.shape}
     
    Shape of y_train {y_train.shape}
    Shape of y_val {y_val.shape}
    Shape of y_test {y_test.shape}
    
    '''
    
    return x_train, x_val, x_test, y_train, y_val, y_test, desc



x_train, x_val, x_test, y_train, y_val, y_test, desc = split_train_val_test(x,y,.5,.3,.2)
print (desc)

Splitting Discription:
    Training Data 50%
    Validation Data 30%
    Testing Data 30%
    
    Shape of Training Data (568630, 30)
    Shape of Target Data (568630, 1)
    
    Shape of x_train (284315, 30)
    Shape of x_val (170589, 30)
    Shape of x_test (113726, 30)
     
    Shape of y_train (284315, 1)
    Shape of y_val (170589, 1)
    Shape of y_test (113726, 1)
    
    


### Normalization

In [111]:
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
x_val = np.asarray(x_val).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
y_val = np.asarray(y_val).astype('float32')



mean = x_train.mean(axis=0)
x_train -= mean

std = x_train.std(axis=0)

x_test -= mean
x_test /= std

### Creating Model

In [112]:
from keras import models
from keras import layers 

model = Sequential()
model.add(Dense(10, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(6, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])


### Training

In [113]:
history = model.fit(x_train, y_train,epochs=100, validation_data =(x_val, y_val))

Epoch 1/100
8885/8885 [==============================] - 16s 2ms/step - loss: 77.5211 - acc: 0.9604 - val_loss: 16.6698 - val_acc: 0.0029
Epoch 2/100
8885/8885 [==============================] - 15s 2ms/step - loss: 0.2570 - acc: 0.9973 - val_loss: 123.8370 - val_acc: 0.0029
Epoch 3/100
8885/8885 [==============================] - 15s 2ms/step - loss: 0.1455 - acc: 0.9977 - val_loss: 52.6060 - val_acc: 0.0029
Epoch 4/100
8885/8885 [==============================] - 15s 2ms/step - loss: 0.0466 - acc: 0.9975 - val_loss: 20.6639 - val_acc: 0.0029
Epoch 5/100
8885/8885 [==============================] - 15s 2ms/step - loss: 0.0295 - acc: 0.9975 - val_loss: 21.7852 - val_acc: 0.0029
Epoch 6/100
8885/8885 [==============================] - 15s 2ms/step - loss: 0.0267 - acc: 0.9976 - val_loss: 24.3893 - val_acc: 0.0029
Epoch 7/100
8885/8885 [==============================] - 15s 2ms/step - loss: 0.0303 - acc: 0.9977 - val_loss: 23.0455 - val_acc: 0.0029
Epoch 8/100
8885/8885 [================

### Evaluation on Test Data

In [114]:
loss, acc = model.evaluate(x_test, y_test)

3554/3554 [==============================] - 3s 873us/step - loss: 13.5805 - acc: 0.0000e+00


### Prediction

In [116]:
pred = model.predict(x_test[0:5])
pred

array([[1.1580802e-06],
       [1.1580802e-06],
       [1.1580802e-06],
       [1.1580802e-06],
       [1.1580802e-06]], dtype=float32)